### PyFVCOM plotting tools examples

Here, we take a transect through the water column temperature or salinity dato and plot it. 

We load some model output into an object which can be passed to a number of plotting objects. These objects have methods for plotting different aspects of the data.

In [25]:
%matplotlib inline

In [26]:
# Load an FVCOM model output and plot a surface.
from PyFVCOM.read import FileReader
from PyFVCOM.plot import Plotter, Depth
#from PyFVCOM.tide import make_water_column
from cmocean import cm
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
# Load an FVCOM model output and plot a surface.
#from PyFVCOM.read import FileReader
#from PyFVCOM.plot import Plotter, Time, Depth
#from PyFVCOM.tide import make_water_column
#from cmocean import cm
#import matplotlib.pyplot as plt
import mpl_toolkits
from IPython import display
import os, time
import numpy as np

### Time snaps to plot. 
We set this to load only those snaps, and not all time records from the FVCOM output file. 
If we set snaps = False, all the time records will be loaded.

We can set the variable to_file = True to save the plots directly to some defined location. 

In [27]:
# Set time snaps to plot
snaps = True
snap1 = 100
snap2 = 100
#variable to plot in the transect ('temp', 'salinity')
var_plot = 'salinity'

# Set the option to display the plots (to_file = False) or to save to a file (to_file = True)
to_file = False
path_to_plots = r"W:\users\odiaz\models\FVCOM\intel_2022.0.2\fvcom27\run_PB\OUTPUT_PB10.1_test\netcdf\plot_transect"

if to_file:
    if not os.path.exists(path_to_plots):
        try:
            os.makedirs(path_to_plots)
        except:
            print("Can't create folder: "+path_to_plots)
            to_file = False


### FVCOM file to read

In [28]:
# Create an object which holds the model outputs. We're only loading
# bathimetry to plot it. 
mydir = r"W:\users\odiaz\models\FVCOM\intel_2022.0.2\fvcom27\run_PB\OUTPUT_PB10.1_test\netcdf"
myfile = 'pb10.1_0001.nc'
#mydir = r"W:\users\odiaz\models\FVCOM\intel_2022.0.2\fvcom27\run_MissRiverWedge\OUTDIR_mrp07#PBS\netcdf"
#myfile = 'mrp07#PBS_0001.nc'
if snaps:
    fvcom = FileReader( os.path.join(mydir, myfile), dims={'time': range(snap1, snap2)}, variables=['time','h',var_plot])
else:
    fvcom = FileReader( os.path.join(mydir, myfile), variables=['time','h',var_plot])

"is" with a literal. Did you mean "=="? (c:\tools\github\pyfvcom\PyFVCOM\read.py:1824)
"is" with a literal. Did you mean "=="? (c:\tools\github\pyfvcom\PyFVCOM\read.py:1829)
"is" with a literal. Did you mean "=="? (c:\tools\github\pyfvcom\PyFVCOM\read.py:1824)
"is" with a literal. Did you mean "=="? (c:\tools\github\pyfvcom\PyFVCOM\read.py:1829)
"is" with a literal. Did you mean "=="? (c:\tools\github\pyfvcom\PyFVCOM\read.py:1824)
"is" with a literal. Did you mean "=="? (c:\tools\github\pyfvcom\PyFVCOM\read.py:1829)
"is" with a literal. Did you mean "=="? (c:\tools\github\pyfvcom\PyFVCOM\read.py:1824)
"is" with a literal. Did you mean "=="? (c:\tools\github\pyfvcom\PyFVCOM\read.py:1829)
"is" with a literal. Did you mean "=="? (c:\tools\github\pyfvcom\PyFVCOM\read.py:1824)
"is" with a literal. Did you mean "=="? (c:\tools\github\pyfvcom\PyFVCOM\read.py:1829)
"is" with a literal. Did you mean "=="? (c:\tools\github\pyfvcom\PyFVCOM\read.py:1824)
"is" with a literal. Did you mean "=="? (c:

TypeError: strptime() argument 1 must be str, not range

### Select the transect to plot

In [ ]:
# Make a plot of the topo/bathymetry.
num = 2  # number of points for the transect (2 is the only available option now)
#positions = np.zeros(shape=(2,2))
#positions[0,:] = [x1 , y1]
#positions[1,:] = [x2 , y2]
#positions = np.array(((-5, 50), (-4.5, 49.5)))

fig1 = plt.figure(1)
plot = Plotter(fvcom, figure=fig1,
               figsize=(20, 20),  
               cartesian=True,             
               cb_label='{} ({})'.format(fvcom.atts.h.long_name,
                                         fvcom.atts.h.units),
               cmap='jet')
plot.plot_field(fvcom.data.h[:], cmap='jet')
plot.axes.set_title('Topo/Bathymetry (m)')

x = fig1.ginput(num)
print(x)

display.display(plt.gcf()) 

In [ ]:
# Plot a temperature transect between two locations.
positions = np.asarray(x)
indices, distance = fvcom.horizontal_transect_nodes(positions)

fig2 = plt.figure(2)
plot = Depth(fvcom, figure=fig2, figsize=(20, 10), 
#             cb_label='Temperature ({})'.format(fvcom.ds.variables['temp'].units),
             cb_label='Salinity ({})'.format(fvcom.ds.variables['salinity'].units),
             cmap=cm.thermal)
# fill_seabed makes the part of the plot below the seabed grey.

#plot.plot_slice(distance / 1000,  # to kilometres from metres
#                fvcom.grid.siglay_z[:, indices],
#                fvcom.data.salinity[4, :, indices],
#                fill_seabed=True)

X = distance / 1000  # to kilometers from meters
Y = fvcom.grid.siglay_z[:, indices]
for i in range(len(fvcom.data.time[:])):
    display.clear_output(wait=True)
    Z = fvcom.data.salinity[i, :, indices].T

    plot.plot_slice(X, Y, Z, fill_seabed=True)
    plot.axes.set_xlim(right=(distance / 1000).max())  # set the x-axis to the data range
    plot.axes.set_xlabel('Distance (km)')
    plot.axes.set_ylabel('Depth (m)')
    
    date_stamp = fvcom.time.datetime[i].strftime('%Y%m%d%H%M%S')
    date_stamp_title = fvcom.time.datetime[i].strftime('%Y-%m-%d %H:%M:%S')    

    figname = "fvcom_plot_transect_"+var_plot+"_"+date_stamp+".png"
    print(figname)

    if to_file:
        plot.figure.savefig(os.path.join(path_to_plots, figname), bbox_inches='tight')
    else:
        display.display(plt.gcf())        
        
        time.sleep(0.1)

